In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets accelerate peft bitsandbytes sentencepiece
!pip install evaluate
!pip install nltk
!pip install rouge_score


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=25026 sha256=8c783c3cdcf1993138ee7cc3b955a4d95ade874359f5d9f492c8162146593b98
  Stored in directory: c:\users\christophers\appdata\local\pip\cache\wheels\44\af\da\5ffc433e2786f0b1a9c6f458d5fb8f611d8eb332387f18698f
Successfully built rouge_score



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.distributed as dist

def setup_distributed():
    dist.init_process_group("nccl") 
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)

if torch.cuda.device_count() > 1:
    setup_distributed()

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available. PyTorch will use the GPU.')
else:
    device = torch.device('cpu')
    print('CUDA is not available. PyTorch will use the CPU.')

CUDA is available. PyTorch will use the GPU.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

# Load DeepSeek model in 8-bit mode
model_name = "deepseek-ai/deepseek-llm-7b-base" 

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": int(os.environ["LOCAL_RANK"])}  # Assign model to the correct GPU
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Apply LoRA for low-rank adaptation
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # LoRA applied to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

if torch.cuda.device_count() > 1:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[int(os.environ["LOCAL_RANK"])])

model.print_trainable_parameters()

pytorch_model-00002-of-00002.bin:  15%|#4        | 566M/3.85G [00:00<?, ?B/s]

C:\Users\ChristopherS\Desktop\lora-finetuning\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ChristopherS\.cache\huggingface\hub\models--deepseek-ai--deepseek-llm-7b-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


In [31]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load SciTLDR dataset (FullText version)
dataset = load_dataset("allenai/scitldr", "FullText")

# Tokenization function
def tokenize_function(examples):
    # Convert list of sentences into a single string
    inputs = ["Prompt: " + " ".join(doc) for doc in examples["source"]]
    targets = [" ".join(summary) if isinstance(summary, list) else summary for summary in examples["target"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1992 [00:00<?, ? examples/s]

Map:   0%|          | 0/618 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

In [11]:
import evaluate
from nltk.tokenize import word_tokenize

# Load evaluation metrics
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

# Define a function to compute metrics
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE evaluation
    rouge_scores = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    
    # METEOR evaluation
    meteor_scores = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"],
        "meteor": meteor_scores["meteor"]
    }


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ChristopherS\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ChristopherS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ChristopherS\AppData\Roaming\nltk_data...


In [ ]:
training_args = TrainingArguments(
    output_dir="./deepseek_scitldr_lora",
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,  # Adjusted for multiple GPUs
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,  # Mixed precision training
    push_to_hub=False,
    save_total_limit=2,
    load_best_model_at_end=True,
    label_names=["labels"],
    dataloader_drop_last=True,  # Ensure even distribution across GPUs
    gradient_accumulation_steps=4,  # Helps with memory efficiency
    report_to="none"  # Prevents logging issues in multi-GPU training
)


In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\ChristopherS\AppData\Local\Temp\ipykernel_13700\2218746015.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate(tokenized_datasets["test"])
print("Final Test Evaluation:")
print(results)


In [ ]:
model.save_pretrained("./deepseek_scitldr_lora")
tokenizer.save_pretrained("./deepseek_scitldr_lora")


In [ ]:
# ---------
model = AutoModelForCausalLM.from_pretrained("./deepseek_scitldr_lora")
tokenizer = AutoTokenizer.from_pretrained("./deepseek_scitldr_lora")
input_text = "Summarize: Deep learning has achieved state-of-the-art results in many NLP tasks."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda()
output = model.generate(input_ids, max_length=128)
print(tokenizer.decode(output[0], skip_special_tokens=True))

huggingface-cli login  # Log in first
model.push_to_hub("your_username/deepseek_scitldr_lora")
